In [8]:
#import pysr
import pandas as pd
import os
from pysr import PySRRegressor,TensorBoardLoggerSpec
import sympy
import numpy as np
import itertools
import random
import ast
import re
import warnings
warnings.filterwarnings("ignore")

In [9]:
# create dataframes 
dir = "datasets/Feynman/Feynman_with_units"
df_dict={}
for file in os.listdir(dir):
    fname = os.fsdecode(file)
    df = pd.read_csv(os.path.join(dir,fname),sep=" ",header=None,engine='pyarrow') # switching engine to pyarrow sped up loading by nearly 6x
    dfname,_ = os.path.splitext(fname)
    df_dict[dfname] = df

print("DFs created")
file="best_equations.csv",  
best_list=[]
c=0


DFs created


In [ ]:
from pydantic import BaseModel,Field,field_validator
import typing

toggle_strict_validation=False
# check if it is not math ???
class ResponseStructure(BaseModel):
    equations:typing.List[str] = Field(...,min_length=1,max_length=3,description="A list of equations generated by the model")
    @field_validator("equations")
    @classmethod
    def check_equations(cls,expr_list):
        for expr in expr_list:
            try:
                ast.parse(expr,mode='eval')
            except SyntaxError as e:
                raise ValueError(f"Syntax error in equation '{expr}': {e}")
        return expr_list
        

                
        
schema = ResponseStructure.model_json_schema()

In [ ]:
import openai
import typing

df_meta = pd.read_csv('datasets/Feynman/FeynmanEquations.csv') # get the number of variables, the variable names 
df_units = pd.read_csv('datasets/Feynman/units.csv')
unit_map = df_units.set_index('Variable')['Units'].to_dict()

def call_model(schema:dict[str,typing.Any],
              equations:pd.DataFrame,
              meta:pd.DataFrame,
              df_name:str,
              MAX_ATTEMPTS:int=20,
              MAX_REQ_EXPRS:int=3)->typing.List[str]:   

    client = openai.OpenAI(
        base_url="http://localhost:8080/v1", # 
        api_key = "sk-no-key-required"
    )
    parsed=None
    out='{}' # empty JSON object.

    num_vars=meta.loc[meta['Filename'] == df_name, '# variables'].values[0]
    print(num_vars)
    variables= [meta.loc[meta['Filename'] == df_name,f'v{i+1}_name'].values[0] for i in range(int(num_vars))]
    print(variables)
    current_units = {v: unit_map.get(v) for v in variables}
    print(current_units)    
    mapping={var:f'x{i}' for i,var in enumerate(variables)}
    print(mapping)

    correct_expr_list=[]
    wrong_expr_list=[]
    attempts=0
    while len(correct_expr_list)<=MAX_REQ_EXPRS:
        if attempts==MAX_ATTEMPTS:
            break  

        completion = client.chat.completions.create(
            model="local-model",
            messages=[
                {
                    "role": "system",
                    "content": "You are an expert at crafting equations."
                },
                {
                    "role": "user",
                    "content": f'''
                                The following are equations generated by an evolutionary search algorithm. 
                                Given these equations, you will generate three more NEW equations by combining 
                                any of the given equations in any way. The generated equations MUST be valid Julia
                                syntax and MUST be enclosd in a python list of strings (eg: ["juliaexpr1", "juliaexpr2", "juliaexpr3"]) 
                                
                                The following is additional context about the equations:                               
                                number of variables: {int(num_vars)}
                                variables: {variables}
                                units: {current_units}
                                
                                make sure to use ONLY the above mentioned variable names

                                                                
                                {equations[['complexity','loss','equation','score']]}                      
                                '''
                
                } # markdown chosen since benchmarks show it uses 40% less tokens than JSON (explore others?)  
            ],
            response_format={
                "type": "json_schema",
                "json_schema": {
                    "name": "response",
                    "schema": schema
                }
            },
            temperature=0.3,
            
        )
        
        try:
            out=completion.choices[0].message.content
            print(f"validating {out} ... ")
            parsed = ResponseStructure.model_validate_json(out)
            print("validated successfully")
            for eq in parsed.equations:
                if eq not in correct_expr_list:
                    correct_expr_list.append(eq)
            print("Current Valid Equations: ", correct_expr_list)     
              
        except Exception as e:
            wrong_expr_list.append(out)
            print("Wrong Expr List: ", wrong_expr_list)        
            print(f"Validation failed with {e}. Retrying. Attempt# {attempts+1}")
        attempts+=1
    
    for i,expr in enumerate(correct_expr_list):
        for var,n_var in mapping.items():
            pattern= r'\b' + re.escape(var) + r'\b'
            expr = re.sub(pattern, n_var, expr)
        correct_expr_list[i] = expr
        
    print("final expression list: ")
    print(correct_expr_list)
    return correct_expr_list
    



In [6]:
equations = pd.read_pickle('equations_df.pkl')
print("final:", call_model(schema,equations,df_meta,'III.12.43'))

2.0
['n', 'h']
{'n': 'Dimensionless', 'h': 'Angular momentum'}
{'n': 'x_0', 'h': 'x_1'}
{"equations": ["2.6061735", "x0 * x0", "(x0 * x0) + -1.9339161", "(x0 * (x0 + 0.09836235)) - 1.9156379", "(cos(x4) + -2.0155823) + (x0 * x0)"]}

    
validating...
validated successfully
Current Valid Equations:  ['2.6061735', 'x0 * x0', '(x0 * x0) + -1.9339161', '(x0 * (x0 + 0.09836235)) - 1.9156379', '(cos(x4) + -2.0155823) + (x0 * x0)']
final: ['2.6061735', 'x0 * x0', '(x0 * x0) + -1.9339161', '(x0 * (x0 + 0.09836235)) - 1.9156379', '(cos(x4) + -2.0155823) + (x0 * x0)']


In [10]:
equations[['complexity','loss','equation','score']].to_markdown()

'|    |   complexity |        loss | equation                                                              |      score |\n|---:|-------------:|------------:|:----------------------------------------------------------------------|-----------:|\n|  0 |            1 | 34.6187     | 2.6061735                                                             | 0          |\n|  1 |            3 |  5.59095    | x0 * x0                                                               | 0.911623   |\n|  2 |            5 |  1.85054    | (x0 * x0) + -1.9339161                                                | 0.552836   |\n|  3 |            7 |  1.82919    | (x0 * (x0 + 0.09836235)) - 1.9156379                                  | 0.00580157 |\n|  4 |            8 |  0.464995   | (cos(x4) + -2.0155823) + (x0 * x0)                                    | 1.3696     |\n|  5 |            9 |  0.3803     | exp(cos(x4)) + ((x0 * x0) + -3.304169)                                | 0.201065   |\n|  6 |           11 |  

In [ ]:
# main loop
# notes..how do I terminate the search when it finds out the ""best"" equation ? 

MAX_CALLS=9
c=0
for name,df in df_dict.items(): # try for 5 equatoins first. 
    if c==5:
        break
    c+=1
    X=df.iloc[:10000,:-1]
    y=df.iloc[:10000,-1] 
    print(f"Going through DF {c} solving for equation {name}, X shape: {X.shape}, y shape: {y.shape}")
    logger_spec = TensorBoardLoggerSpec(
                    log_dir="logs/run",
                    log_interval=5,  # Log every 5 iterations
                    )               
    model=PySRRegressor(niterations=5,
                        warm_start=True,
                        guesses=[],
                        batching=True,
                        batch_size=32,
                        verbosity=0,
                        constraints={"sin": 10, "cos": 10,
                                    "exp": 20, "log": 20, "sqrt": 20,
                                    "pow": (-1, 20)},
                        nested_constraints={"sin": {"sin": 0, "cos": 0},
                                            "cos": {"sin": 0, "cos": 0},
                                            "exp": {"exp":0, "log": 0},
                                            "log": {"exp": 0, "log": 0},
                                            "sqrt": {"sqrt": 0}},
                        unary_operators=["sqrt","exp","sin","cos","log"],
                        binary_operators=["+", "*", "-", "/", "^"],
                        logger_spec=logger_spec)
    
    

    
    model.fit(X,y)
    for _ in range (MAX_CALLS):
        randfloat=random.random()
        print(randfloat)
        if randfloat<0.1: # 10% of the time
            print(f"LLM called. Sending equations and meta info")
            model.guesses = call_model(schema,model.equations_,df_meta,name)   
            model.fit(X,y)
        else:
            # continue as normal 
            model.fit(X,y)

    best=model.equations_.sort_values("score",axis=0,ascending=False).iloc[[0]] # score chooses the equation with the best complexity-accuracy trade off.
    best["name"] = name 
    best_list.append(best)
    
# now concat vertically and save
pd.concat(best_list,axis=0).to_csv('best_eqs.csv',index=False,header=["complexity","loss","equation","score","sympy format","lambda_format","Filename"])
 

Going through DF 1 solving for equation III.15.27, X shape: (10000, 3), y shape: (10000,)
0.3759998009799974
0.031589645247976805
LLM called. Sending equations and meta info
3.0
['alpha', 'n', 'd']
{'alpha': 'Dimensionless', 'n': 'Dimensionless', 'd': 'Length'}
{'alpha': 'x0', 'n': 'x1', 'd': 'x2'}
validating {out} ... 
validated successfully
Current Valid Equations:  ['alpha * n^d', 'sin(n) + alpha', 'alpha / n * 2.4532824']
validating {out} ... 
validated successfully
Current Valid Equations:  ['alpha * n^d', 'sin(n) + alpha', 'alpha / n * 2.4532824', 'alpha * n / d']
final expression list: 
['x0 * x1^x2', 'sin(x1) + x0', 'x0 / x1 * 2.4532824', 'x0 * x1 / x2']
0.6323204438977056
0.14785569203823068
0.6881998522386378
0.1914844729283075
0.4781021716712346
0.4743597519506352
0.0692518212142772
LLM called. Sending equations and meta info
3.0
['alpha', 'n', 'd']
{'alpha': 'Dimensionless', 'n': 'Dimensionless', 'd': 'Length'}
{'alpha': 'x0', 'n': 'x1', 'd': 'x2'}
validating {out} ... 
val

In [12]:
symbols = sympy.symbols("""
sigma theta1 x1 x2 y1 y2 m1 m2 G z1
z2 m_0 v c x3 y3 mu Nn q1 q2
epsilon r Ef q B m u w r1 r2
z k_spring x t F omega omega_0 n theta2 d1
d2 Int_0 lambd d a p h I1 I2 delta
pr gamma kb n_0 mu_drift Volt mob V1 V2 rho
alpha kappa T1 T2 Pwr p_d y sigma_den chi n_rho
I rho_c_0 mom g_ Jz E_n Bx By Bz k
I_0 beta A_vec x0 x4 x5 x6 x7 x8 x9
""")
(sigma, theta1, x1, x2, y1, y2, m1, m2, G, z1,
z2, m_0, v, c, x3, y3, mu, Nn, q1, q2,
epsilon, r, Ef, q, B, m, u, w, r1, r2,
z, k_spring, x, t, F, omega, omega_0, n, theta2, d1,
d2, Int_0, lambd, d, a, p, h, I1, I2, delta,
pr, gamma, kb, n_0, mu_drift, Volt, mob, V1, V2, rho,
alpha, kappa, T1, T2, Pwr, p_d, y, sigma_den, chi, n_rho,
I, rho_c_0, mom, g_, Jz, E_n, Bx, By, Bz, k,
I_0, beta, A_vec,x0,x4,x5,x6,x7,x8,x9) = symbols

locals_ = {s.name: s for s in symbols}


In [ ]:
# creating the core DF 

df_pred=pd.read_csv("best_eqs.csv")
df_original = pd.read_csv("~/projects/FeynmanEquations.csv")[["Filename","Formula"]]
df_merged = df_pred.merge(df_original,on="Filename",how='left')
df_merged["Equation"] = df_merged["Formula"].map(lambda f: sympy.sympify(f, locals=locals_))
df_merged["sympy format"] = df_merged["sympy format"].map(lambda f: sympy.sympify(f, locals=locals_))
df_merged= df_merged[["Filename","Equation","loss","sympy format"]]
df_merged.set_index("Filename")
df_merged.drop(df_merged[df_merged['Equation']==sympy.nan].index,inplace=True)




In [14]:
df_merged

,Filename,Equation,loss,sympy format
0,III.15.27,2*pi*alpha/(d*n),8.327730e-14,6.2831855*x0/(x1*x2)
1,III.4.33,h*omega/(2*pi*(exp(h*omega/(2*pi*T*kb)) - 1)),6.020358e-01,x2*x3
2,I.24.6,m*x**2*(omega**2 + omega_0**2)/4,6.909887e+00,x0*x3**2*(x2 - cos(x1))
3,III.9.52,8*pi*Ef*p_d*sin(t*(omega - omega_0)/2)**2/(h*t...,2.744946e+01,4.81302122383196*x0*x1*exp(cos(x4 - x5))/x3
4,I.34.14,omega_0*(1 + v/c)/sqrt(1 - v**2/c**2),1.010983e-02,x2*exp(x1/x0)
...,...,...,...,...
95,II.10.9,sigma_den/(epsilon*(chi + 1)),7.702505e-16,x0/(x1*(x2 + 1.0))
96,III.12.43,h*n/(2*pi),1.330285e-14,0.15915494*x0*x1
97,II.27.16,Ef**2*c*epsilon,1.120711e-10,x0*x1*x2**2
98,I.47.23,sqrt(gamma*pr/rho),7.385825e-15,sqrt(x0*x1/x2)


In [15]:
# i want to know how similar these are...
# constants are an issue.. we will try to prove equivalence up to a constant 
pred_eqs = df_merged['sympy format']
orig_eqs = df_merged['Equation']


In [ ]:
def check_equality(pred_expr, true_expr, true_varnames=None):
    '''
    used to check if two sympy expressions are "equal".
    equality is defined here as equal upto a constant.
    Works by sampling the true equation and then trying out permutations 
    of the input variables of the candidate solution and obtaining the ratio of the fits. 
    If the ratios are identical, the expressions are equivalent upto a constant.  
    
    '''
    # print("Free symbols in the true expression: ")
    # print(true_expr.free_symbols)
    pred_vars = sorted(pred_expr.free_symbols, key=lambda s: s.name)
    if true_varnames:
        true_vars = [sympy.Symbol(n) for n in true_varnames]
    else:
        true_vars = sorted(true_expr.free_symbols, key=lambda s: s.name)
    # print("variables in the true expression: ")
    print(true_vars)
    if len(pred_vars) != len(true_vars):
        return False
# lambdify
    f_pred = sympy.lambdify(pred_vars, pred_expr, "numpy")
    f_true = sympy.lambdify(true_vars, true_expr, "numpy")
    
# 10 points 
    N_samples = 10
    X = np.random.uniform(1.0, 5.0, (N_samples, len(true_vars)))
    
    # Calculate True Output
    try:
        y_true = f_true(*X.T) # evaluate true function over 10 points. 
    except Exception:
        return False 

    best_perm = None
    
    for perm_indices in itertools.permutations(range(len(true_vars))): # [0,1,2] eg if len is 3
        print(perm_indices) # [0,1,2], [0,2,1]
        try:
            # Reorder columns of X to match this permutation
            X_permuted = X[:, perm_indices]
            
            # Eval prediction
            y_pred = f_pred(*X_permuted.T) # get the fit on y after re-ordering. 
            
            # Calculate Ratio: y_pred / y_true
            # If equations are equivalent (up to constant), ratio should be identical for all points
            ratio = y_pred / y_true            
            # Check if ratio is constant (sigma is almost 0)
            if np.std(ratio) < 1e-5 and np.isfinite(ratio).all():
                # We found a candidate permutation! 
                # Map the indices back to symbols
                best_perm = [true_vars[i] for i in perm_indices]
                break
                
        except Exception:
            continue

    if best_perm:
        mapper = dict(zip(pred_vars, best_perm)) # make the correct mapping.
        pred_mapped = pred_expr.subs(mapper)
        
        # run simplify for a final sanity check 
        final_ratio = sympy.simplify(pred_mapped / true_expr)
        
        if final_ratio.is_constant():
            return True

    return False

In [ ]:
# finally, apply the above onto the dataframe and add the col
df_merged["equivalent"] = df_merged.apply(
    lambda row: check_equality(
        row["sympy format"],   # predicted
        row["Equation"]        # ground truth
    ),
    axis=1
)



In [18]:
df_merged[df_merged['equivalent']==True]


,Filename,Equation,loss,sympy format,equivalent
0,III.15.27,2*pi*alpha/(d*n),8.327730e-14,6.2831855*x0/(x1*x2),True
5,III.7.38,4*pi*B*mom/h,1.810693e-11,12.566371*x0*x1/x2,True
8,I.18.12,F*r*sin(theta),3.964229e-13,x0*x1*sin(x2),True
10,II.21.32,q/(4*pi*epsilon*r*(1 - v/c)),1.235921e-16,0.07957746*x0/(x1*x2*(-x3/x4 + 1.0)),True
11,II.6.15b,3*p_d*sin(theta)*cos(theta)/(4*pi*epsilon*r**3),2.287197e-17,0.23873241*x1*sin(x2)*cos(x2)/(x0*x3**3),True
12,II.34.29b,2*pi*B*Jz*g_*mom/h,7.030817e-10,6.2831855*x0*x2*x3*x4/x1,True
13,I.14.3,g*m*z,4.462443e-12,x0*x1*x2,True
14,I.13.12,G*m1*m2*(1/r2 - 1/r1),7.885416e-13,x0*x1*x4*(x2 - x3)/(x2*x3),True
16,I.12.5,Ef*q2,2.798131e-13,x0*x1,True
17,I.43.16,Volt*mu_drift*q/d,1.221977e-12,x0*x1*x2/x3,True


In [ ]:
df_merged.shape
# score (pysr with defaults) = 46/97 (appx 47%) # took at most a minute to run
# score (pysr with added sqrt, sin, exp, cos) = 55/97 (appx 56%) ~ took about 30 minutes to run

(97, 5)

{
"answer": "def black_body_radiation(T):\n    return 8.6173303e-11 * T**3"
}

  
validating...
validated successfully
Correct Expr List:  {'black_body_radiation': 'def black_body_radiation(T):\n    return 8.6173303e-11 * T**3'}
{
"answer": "def quantum_mechanics(E, h, epsilon):\n    return (h**2) / (2*E*epsilon)"
}

  
validating...
validated successfully
Correct Expr List:  {'black_body_radiation': 'def black_body_radiation(T):\n    return 8.6173303e-11 * T**3', 'quantum_mechanics': 'def quantum_mechanics(E, h, epsilon):\n    return (h**2) / (2*E*epsilon)'}
{
    "answer": "def black_body_radiation(T, h, c, k):\n    return 2.0 * h * (c**2) / (T**4)"
}

    
validating...
validated successfully
Correct Expr List:  {'black_body_radiation': 'def black_body_radiation(T, h, c, k):\n    return 2.0 * h * (c**2) / (T**4)', 'quantum_mechanics': 'def quantum_mechanics(E, h, epsilon):\n    return (h**2) / (2*E*epsilon)'}
{
    "answer": "def black_body_radiation(T, h, c, k):\n    return 2.0 * h

In [19]:
wrong_expr_list

[]

In [20]:
correct_expr_list

{'black_body_radiation': 'def black_body_radiation(T, h, c, k):\n    return 2*h*c**2/T**5',
 'quantum_mechanics': 'def quantum_mechanics(E, h, epsilon):\n    return (h**2) / (2*E*epsilon)',
 'quantum_dynamics': 'def quantum_dynamics(m, v):\n    return m * v',
 'planck_radiation': 'def planck_radiation(T):\n    import math\n    h = 6.62607004e-34\n    c = 299792458\n    k = 1.38064852e-23\n    return (h * c) / (T * k)\n'}